Mit Datenbank verbinden

In [1]:
from sqlalchemy import create_engine
import json

f = open("credentials.txt")
data = json.load(f)

f.close()

In [2]:
database = data['database']
username = data['username']
password = data['password']
host = data['host']

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')

In [3]:
connection = engine.connect()

Tabelle anlegen

In [4]:
from sqlalchemy import Column, Integer, String, ForeignKey #weitere Datentypen: Date, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

Base = declarative_base()

# Definition der Professor-Tabelle
class Professor(Base):
    __tablename__ = 'professor'
    id = Column(Integer, primary_key=True)  #Alternativen unique=True und default=Defaultwert
    name = Column(String(50)) ## Länge auf 50 Zeichen festgelegt
    vorlesungen = relationship("Vorlesung", back_populates="professor") ##Fremdschlüsselbeziehung --> sorgt dafür dass Änderungen an der einen Seite automatisch auf die andere Seite reflektiert wird wenn die Objekte in einer Session benutzt werden
    ##Bei 1:1 Beziehung könnte man in der relationship-Methode noch den uselist Parameter auf False setzen (Beziehung enthält dann nur 1 einziges Objekt)

# Definition der Vorlesung-Tabelle
class Vorlesung(Base):
    __tablename__ = 'vorlesung'
    id = Column(Integer, primary_key=True)
    titel = Column(String)
    professor_id = Column(Integer, ForeignKey('professor.id'))
    professor = relationship("Professor", back_populates="vorlesungen")

# Erstellen der Tabellen in der Datenbank, falls sie noch nicht existieren
Base.metadata.create_all(engine)

C:\Users\timm.gieger\AppData\Local\Temp\ipykernel_17016\3350511582.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Alle Professoren:
1: Dr. Müller
2: Prof. Schmidt

Vorlesungen mit zugehörigen Professoren:
1: Mathematik I - Professor: Dr. Müller
2: Physik I - Professor: Dr. Müller
3: Informatik II - Professor: Prof. Schmidt


Einträge einfügen und abfragen

In [ ]:
## Einträge einfügen
if __name__ == "__main__":
    # Erstellen einer Session
    Session = sessionmaker(bind=engine)
    session = Session()

    # Erstellen von Beispiel-Datensätzen
    professor1 = Professor(name='Dr. Müller')
    professor2 = Professor(name='Prof. Schmidt')

    vorlesung1 = Vorlesung(titel='Mathematik I', professor=professor1)
    vorlesung2 = Vorlesung(titel='Informatik II', professor=professor2)
    vorlesung3 = Vorlesung(titel='Physik I', professor=professor1)

    # Hinzufügen der Datensätze zur Session
    session.add_all([professor1, professor2, vorlesung1, vorlesung2, vorlesung3])
    session.commit()

    # Beispielabfragen
    print("Alle Professoren:")
    for professor in session.query(Professor):
        print(f"{professor.id}: {professor.name}")

    print("\nVorlesungen mit zugehörigen Professoren:")
    for vorlesung in session.query(Vorlesung):
        print(f"{vorlesung.id}: {vorlesung.titel} - Professor: {vorlesung.professor.name}")

    # Session schließen
    session.close()

In [5]:
## Erstellen eines neuen Profs, Löschen einer Vorlesung und ändern eines Eintrags
if __name__ == "__main__":
    # Erstellen einer Session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Vorlesung "Mathematik I" löschen
        mathematik_vorlesung = session.query(Vorlesung).filter_by(titel='Mathematik I').first() ##ebenfalls .all(), one() - löst Exception aus falls mehrere zurückkommen
        if mathematik_vorlesung:
            session.delete(mathematik_vorlesung)
            session.commit()
            print(f"Vorlesung '{mathematik_vorlesung.titel}' gelöscht.")

        # Neuen Professor für die Vorlesung "Physik I" erstellen
        physik_vorlesung = session.query(Vorlesung).filter_by(titel='Physik I').first()
        if physik_vorlesung:
            neuer_professor = Professor(name='Prof. Schneider')
            physik_vorlesung.professor = neuer_professor
            session.add(neuer_professor)
            session.commit()
            print(f"Neuer Professor '{neuer_professor.name}' für die Vorlesung '{physik_vorlesung.titel}' eingetragen.")
    
    except Exception as e:
        session.rollback()
        print(f"Fehler beim Bearbeiten der Datenbank: {e}")

    finally:
        # Session schließen
        session.close()

Vorlesung 'Mathematik I' gelöscht.
Neuer Professor 'Prof. Schneider' für die Vorlesung 'Physik I' eingetragen.


Filtering mit or / and

In [8]:
from sqlalchemy import or_ , and_

if __name__ == "__main__":
    # Erstellen einer Session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Beispielabfrage mit or_()
        vorlesung_math1 = Vorlesung(titel="Mathematik I", professor=professor1)
        # Hinzufügen der Datensätze zur Session
        session.add_all([vorlesung_math1])
        session.commit()

        # Alle Vorlesungen von Professor Dr. Müller oder Prof. Schmidt
        results_or = session.query(Vorlesung).join(Vorlesung.professor).filter(or_(Professor.name == 'Dr. Müller', Professor.name == 'Prof. Schmidt')).all() #Verwendung von join ist optional
        print("Ergebnisse der OR-Abfrage:")
        for vorlesung in results_or:
            print(f"Vorlesung: {vorlesung.titel}, Professor: {vorlesung.professor.name}")

        # Beispielabfrage mit and_()

        # Alle Vorlesungen von Professor Dr. Müller, die Mathematik I oder Physik I sind
        results_and = session.query(Vorlesung).join(Vorlesung.professor).filter(and_(Professor.name == 'Dr. Müller', or_(Vorlesung.titel == 'Mathematik I', Vorlesung.titel == 'Physik I'))).all()
        print("\nErgebnisse der AND-Abfrage:")
        for vorlesung in results_and:
            print(f"Vorlesung: {vorlesung.titel}, Professor: {vorlesung.professor.name}")

    except Exception as e:
        session.rollback()
        print(f"Fehler beim Bearbeiten der Datenbank: {e}")

    finally:
        # Session schließen
        session.close()

Ergebnisse der OR-Abfrage:
Vorlesung: Informatik II, Professor: Prof. Schmidt
Vorlesung: Mathematik I, Professor: Dr. Müller

Ergebnisse der AND-Abfrage:
Vorlesung: Mathematik I, Professor: Dr. Müller
